<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [20]:
""" 
# # Installing Java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
! tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
! pip install -q findspark
# # installing pyspark
! pip install pyspark==3.2.1
 """

' \n# # Installing Java\n! apt-get install openjdk-8-jdk-headless -qq > /dev/null\n# # Dowload Spark\n! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz\n# # Decompressing archives\n! tar xf spark-3.2.1-bin-hadoop2.7.tgz\n# # installing findspark\n! pip install -q findspark\n# # installing pyspark\n! pip install pyspark==3.2.1\n '

In [21]:
""" 
# Importing os library
import os
# # Java envirorment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
 """

' \n# Importing os library\nimport os\n# # Java envirorment variable\nos.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"\n# # Spark envirorment variable\nos.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"\n '

# Better and consistent library that makes the same function as the code above



In [22]:
! pip install pyspark py4j


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Importing modules and pandas library that will be used later.**

In [23]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
# import findspark

# findspark.init()

In [24]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

Gooogle Colab Version

In [25]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [26]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/BrStocksFormated.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [27]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Ebit.parquet"

ebit = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [28]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Stat.parquet"

stat = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [29]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Price.parquet"

price = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [30]:
fille =r'C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Balance.parquet'

balance = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [31]:
# Stocks table
stocks.createOrReplaceTempView("General")
# Ebit table
ebit.createOrReplaceTempView("Ebit")
# Stat table
stat.createOrReplaceTempView("Stat")
# Price table
price.createOrReplaceTempView("Price")
# Balance table
balance.createOrReplaceTempView("Balance")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [32]:
# Already called at Calc Table
Gen = spark.sql("""
SELECT * FROM General
WHERE Moeda != 'USD' AND EBITDA != 0 AND ROA != 0 AND ROE != 0;
""").createOrReplaceTempView("Gen")

In [33]:
#Gen.show()

In [34]:
# Already called at RSCH table
Ebit = spark.sql("""
SELECT * FROM Ebit
where Divida_Liquida != 0;
""").createOrReplaceTempView("Debt")

In [35]:
#Ebit.show()

In [36]:
# Already called at RSCH table
Price = spark.sql("""
SELECT * FROM Price
WHERE valor_de_mercado != 0;
""")#.createOrReplaceTempView("Prices")

In [37]:
Price.show()

+--------+----------------+-----------------+-----------------+
|  Ticker|valor_de_mercado|volume_de_mercado|__index_level_0__|
+--------+----------------+-----------------+-----------------+
|ZAMP3.SA|      1290876288|                0|                0|
|AERI3.SA|       964650304|                0|                3|
|DOTZ3.SA|       113162640|                0|                5|
|MLAS3.SA|      1763443968|                0|                8|
|NINJ3.SA|       134109408|                0|               11|
|MODL3.SA|      1179109888|                0|               14|
|VITT3.SA|      1594519040|                0|               15|
|KRSA3.SA|       874195328|                0|               16|
|ALLD3.SA|       470764032|                0|               23|
|ATMP3.SA|        27776526|                0|               25|
|JSLG3.SA|      1982957056|                0|               27|
|ELMD3.SA|      1802294016|                0|               29|
|OPCT3.SA|       617591488|             

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [40]:
Stat = spark.sql("""
SELECT * FROM Stat
WHERE Beta != 0.5;
""")#.createOrReplaceTempView("Stat")

In [ ]:
#Stat.show()

In [41]:
CALCULATE = spark.sql("""
SELECT * FROM Gen
WHERE Margem_Bruta > 0.0 AND Margem_EBITIDA > 0.0 AND 
Margem_Operacional > 0.0 AND Margem_liquida > 0.0 AND 
ROE > 0 AND Crescimento_de_Receita_3T > 0;
""")#.createOrReplaceTempView("Calc")

In [ ]:
#CALCULATE.show()

In [42]:
RSCH = spark.sql("""
SELECT p.Ticker, p.valor_de_mercado, p.volume_de_mercado, e.EBIT, e.Divida_Liquida FROM Price as p, Ebit as e
WHERE (p.valor_de_mercado + e.Divida_Liquida)/e.EBIT > 0.0 AND 
p.Ticker == e.Ticker; 
""")#.createOrReplaceTempView("RSCH")

In [43]:
Balance = spark.sql("""
SELECT * FROM Balance
""")

In [ ]:
# DFT = spark.sql("""
# SELECT e.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida
# FROM General g
# LEFT JOIN Ebit e ON g.Ticker = e.Ticker
# """).createOrReplaceTempView("Filtered")

In [ ]:
# Stat = spark.sql("""
# SELECT f.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida,
#       Beta,
#       Margem_de_Lucro,
#       Crescimento_de_receitas_4T,
#       Valor_do_ultimo_dividendo
# FROM Filtered f
# LEFT JOIN Stat s ON f.Ticker = s.Ticker;
# """).createOrReplaceTempView("Prstocks")

In [ ]:
# DF = spark.sql("""
# SELECT a.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida,
#        Beta,
#        Margem_de_Lucro,
#        Crescimento_de_receitas_4T,
#        Valor_do_ultimo_dividendo,
#        valor_de_mercado
# FROM Prstocks a
# LEFT JOIN Price p ON a.Ticker = p.Ticker;
# """).createOrReplaceTempView('Stocks')

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

In [ ]:
# df_final = CALCULATE.toPandas()

In [ ]:
# df_final.to_excel("/content/gdrive/MyDrive/Data/XLSX/Final/StocksFinal.xlsx",
#                   sheet_name="Stocks")

Pandas setting DataFrame to pandas format:

In [44]:
df_stat = Stat.toPandas()

df_calc = CALCULATE.toPandas()

df_rsc = RSCH.toPandas()

df_balance = Balance.toPandas()

c:\Users\video\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
c:\Users\video\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\pandas\conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
c:\Users\video\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\pandas\conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

In [45]:
df_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Stat.xlsx",
                 sheet_name="stat")

df_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",
                 sheet_name="calc")

df_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",
                sheet_name="research")

df_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",
                sheet_name="balance")